<a href="https://colab.research.google.com/github/jersict/IBB-Seminar3/blob/main/IBB_Seminar3_TimJer%C5%A1i%C4%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prediction of identity, ethnicity and gender based on images of ears using CNN implementations in TensorFlow and Keras

V tem dokumentu se nahaja moja koda z implementacijo predikcije identitete, rase in spola, glede na analizo ušes. Med različnimi načini delovanja lahko preklapljate s spremenljivko `typeOfClassification`, ki jo nastavite na 0 za napovedovanje identitete, 1 za napovedovanje rase ali 2 za napovedovanje spola.


Sistem ima vnaprej naučene modele z osmimi epochi, za prepoznavanje identitete, rase in spola, ki so shranjeni v datotekah in jih lahko uporabite po želji. To naredite tako, da spremenljivko `Train` nastavite na `False`. 
Opozorilo: Nalaganje modela lahko traja kakšno minuto

V primeru, da želite natrenirati nov model lahko spremenljivko `Train` nastavite na `True`. Prav tako lahko spremenite število epochov za potrebe treniranja v spremenljivki `numEpoch`. 

In [3]:
%cd drive/MyDrive/IBB3/
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
from tensorflow.keras import layers


'''*******  SPREMENLJIVKE *******'''
typeOfClassification = 0  # 0 = Identiteta, 1 = rasa, 2 = spol
Train = False 
Stats = True  
numEpoch = 30        
'''*******  SPREMENLJIVKE *******'''


PATH = os.getcwd()  + '/'
classifications = ['ID', 'Race', 'Gender']
data_dir = PATH + classifications[typeOfClassification]
print("Classification based on", classifications[typeOfClassification])

batch_size = 32
img_height = 180
img_width = 180
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=16,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=16,
  image_size=(img_height, img_width),
  batch_size=batch_size)
class_names = train_ds.class_names
print(class_names)


AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

num_classes = 100
if Train:
  model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(16, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    #layers.Dropout(0.1),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
  ])

  model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

  stats = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=numEpoch
  )
  if Stats:
    import matplotlib.pyplot as plt
    train_loss=stats.history['loss']
    val_loss=stats.history['val_loss']
    train_acc=stats.history['accuracy']
    val_acc=stats.history['val_accuracy']
    xc=range(numEpoch)

    plt.figure(2,figsize=(7,5))
    plt.plot(xc,train_acc)
    plt.plot(xc,val_acc)
    plt.xlabel('num of Epochs')
    plt.ylabel('accuracy')
    plt.title('train_acc vs val_acc')
    plt.grid(True)
    plt.legend(['train','val'],loc=4)
    #print plt.style.available # use bmh, classic,ggplot for big pictures
    plt.style.use(['classic'])
else:
  model = tf.keras.models.load_model('model_'+classifications[typeOfClassification])
predictions = np.argmax(model.predict(val_ds), axis=-1)
predictions = list(predictions)
expected = np.array([])
for i in range(len(list(val_ds))):
  smth = list(val_ds)[i][-1]
  expected = np.concatenate((expected, smth.numpy()))
expected = list(expected)
for i in range(len(expected)):
  expected[i] = round(expected[i])
correct = 0
all = len(expected)
for i in range(len(expected)):
  if (expected[i] == predictions[i]):
    print ("Prediction ", i, ", Predicted: ", predictions[i], " Expected: ", expected[i], " Correct!")
    correct += 1
  else:     
    print ("Prediction ", i, ", Predicted: ", predictions[i], " Expected: ", expected[i], " False!")
print("Correctly predicted ", correct, " out of ", all, " predictions, which accounts to accuracy of ", round(correct*100/all), "%")

[Errno 2] No such file or directory: 'drive/MyDrive/IBB3/'
/content/drive/MyDrive/IBB3
Classification based on ID
Found 1000 files belonging to 100 classes.
Using 800 files for training.
Found 1000 files belonging to 100 classes.
Using 200 files for validation.
['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100']
Prediction  0 , Predicted:  50  Expec